In [ ]:
from geomdl import BSpline
from geomdl import utilities

import numpy as np
import math

from scipy import optimize
from scipy.optimize import minimize, NonlinearConstraint
from scipy.stats import multivariate_normal

from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt

In [ ]:
# FOV_range = 10
# scale_resolution = .1
# x_ = np.arange(0, FOV_range*2, scale_resolution)
# y_ = np.arange(0, FOV_range*2, scale_resolution)
# z_ = np.arange(0, FOV_range*2, scale_resolution)
# x0 = 0
# y0 = 0
# sigx = 1
# sigy = 1
# Xg, Yg = np.meshgrid(x_, y_)
# SiP = [0, 4]
obj = 4
probability_threshold = 0.01
numbers_of_ctrl_points = 20

In [ ]:
mean = np.array([7.0, 7.0, 6.99])
print(mean)
# mean += np.random.normal(0, 1e-3, mean.shape) # add small noise to avoid singular matrix
# mean[2] += np.random.normal(0, 1e-1, mean[2].shape) # add small noise to avoid singular
# print(mean)
cov = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
x = np.linspace(-0.0, 20.0, 50)
y = np.linspace(-0.0, 20.0, 50)
z = np.linspace(-0.0, 20.0, 50)
X, Y, Z = np.meshgrid(x, y, z)
pos = np.empty(X.shape + (3,))
pos[:, :, :, 0] = X
pos[:, :, :, 1] = Y
pos[:, :, :, 2] = Z

In [ ]:
# rv = multivariate_normal(mean, cov)
# W = rv.pdf(pos)

# mean = np.array([-7.0, -7.0, -7.0])
# rv = multivariate_normal(mean, cov)
# W += rv.pdf(pos)

# mean = np.array([7.0, 7.0, 7.0])
# rv = multivariate_normal(mean, cov)
# W += rv.pdf(pos)
# rv = multivariate_normal(mean, cov)

In [ ]:
Start_point = (0.0, 0.0, 0.0) #startpoint
Goal_point = (20.0, 20.0, 20.0) #goal position

In [ ]:
def object_function(x):
    '''Calculate the difference between the goal position and the current
    position of the distance'''
    res = math.dist((x[-3], x[-2], x[-1]), Goal_point)
    # print(res)
    return res

In [ ]:
def con_cltr_pnt(x):
    con = []
    con.append(math.dist((Start_point[0], Start_point[1], Start_point[2]), (x[0], x[1], x[2])))
    for i in range(0, len(x)-3, 3):
        con.append(math.dist([x[i], x[i+1], x[i+2]], [x[i+3], x[i+4], x[i+5]]))
    # print(len(con))
    return con

C_nlc = NonlinearConstraint(con_cltr_pnt, 0.0, 1.0)

In [ ]:
def inturder_linear_path(start, stop, num):
    '''Calculate the linear path between two points'''
    x = np.linspace(start[0], stop[0], num)
    y = np.linspace(start[1], stop[1], num)
    z = np.linspace(start[2], stop[2], num)
    return x, y, z

In [ ]:
# local = inturder_linear_path([0, 15, 10], [16, 10, 10], numbers_of_ctrl_points)
# inturder_list = []
# inturder_list.append(local)
# local = inturder_linear_path([20, 5, 10], [0, 10, 10], numbers_of_ctrl_points)
# inturder_list.append(local)
# local = inturder_linear_path([2.5,0, 10], [20, 17.5, 10], numbers_of_ctrl_points)
# inturder_list.append(local)
# print(inturder_list)

In [ ]:
local = inturder_linear_path([10.1, 10.9, 10.2], [10.1, 10.9, 10.2], numbers_of_ctrl_points)
inturder_list = []
inturder_list.append(local)
local = inturder_linear_path([6, 5, 6.5], [6, 5, 6.5], numbers_of_ctrl_points)
inturder_list.append(local)
local = inturder_linear_path([7.3, 7.5, 9], [7.3, 7.5, 9], numbers_of_ctrl_points)
inturder_list.append(local)

In [ ]:
def con_pdf_muti(x):
    tmp = []

    for i in range(0, len(x), 3):
        out = 0
        for j in range(len(inturder_list)):
            # print(i, j)
            # gau = gauss_function_2d([x[i], x[i+1]], inturder_list[j][0][int(i/2)], inturder_list[j][1][int(i/2)], 1.05**(i/2), 1.05**(i/2))
            mean = np.array([inturder_list[j][0][int(i/3)], inturder_list[j][1][int(i/3)], inturder_list[j][2][int(i/3)]])
            rv = multivariate_normal(mean, cov)
            W = rv.pdf([x[i], x[i+1], x[i+2]])

            out = out + W
        tmp.append(out)
        # print(out)
    return tmp

P_muti_nlc = NonlinearConstraint(con_pdf_muti, 0.0, probability_threshold)

In [ ]:
def con_pdf(x):
    tmp = []    
    for i in range(0, len(x), 3):
        # print(x[i], x[i+1], x[i+2])
        out = rv.pdf([x[i], x[i+1], x[i+2]])
        tmp.append(out)
        # print(out)
    return tmp

P_nlc = NonlinearConstraint(con_pdf, 0.0, probability_threshold)

In [ ]:
int_X0 = np.zeros(numbers_of_ctrl_points*3)
# int_X0 = np.ones(numbers_of_ctrl_points*3)*20
# print(int_X0)
# print(len(int_X0))

In [ ]:
# import numpy as np
# from scipy.optimize import minimize

# myarray = np.asarray([[[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]], [[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]])
# bounds = [(1/9999, 1.0) for i in myarray for j in i for k in j]

# def CostFunction(x):
#     return np.linalg.norm(x)

# out=minimize(fun=CostFunction, x0=myarray.flatten(), method='Powell', bounds=bounds)

In [ ]:
# print(bounds)
# print(len(bounds))

In [ ]:
# res = minimize(object_function, int_X0, method='SLSQP', constraints=[C_nlc, P_nlc])
# res = minimize(object_function, int_X0, constraints=[C_nlc, P_nlc])
res = minimize(object_function, int_X0, constraints=[C_nlc, P_muti_nlc])
# res = minimize(object_function, int_X0, method='BFGS', constraints=[C_nlc, P_nlc])
# res = minimize(object_function, int_X0, bounds=[(-0, 20) for i in range(len(int_X0))], method='SLSQP', constraints=[C_nlc, P_nlc])
print(res.success)
print(res.message)
# print(res.x)
# print(len(res.x))

In [ ]:
# bounds=[(-5, 25) for i in range(len(int_X0))]
# print(bounds)
# print(len(bounds))

In [ ]:
# int_X0 = np.zeros(numbers_of_ctrl_points*3)
# res = minimize(object_function, int_X0, bounds=[(-5, 25) for i in range(len(int_X0))], method='SLSQP', constraints=[nlc, P_nlc])
# print(res.success)
# print(res.message)

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly
plotly.offline.init_notebook_mode()

In [ ]:
# from geomdl import BSpline
# from geomdl import utilities
# import numpy as np
# import plotly.graph_objs as go

# Create a B-Spline curve
curve = BSpline.Curve()
# Set up the curve
curve.degree = 4

x=[]
y=[]
z=[]
x.append(Start_point[0])
y.append(Start_point[1])
z.append(Start_point[2])
for i in range(0, len(res.x), 3):
    # print(res.x[i], res.x[i+1], res.x[i+2])
    x.append(res.x[i])
    y.append(res.x[i+1])
    z.append(res.x[i+2])

# x = np.array(range(10))
# y = np.array(range(10))
# z = np.array(range(10))
# print([x, y, z])
# print(len(x))
cp = np.array([x, y, z]).T.tolist()
# print(cp)
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    name='Control Points',
    marker = dict(
        size = 5
    ),
    opacity=0.5
)
# plt.show()
curve.ctrlpts = cp
# Auto-generate knot vector
curve.knotvector = utilities.generate_knot_vector(curve.degree, len(curve.ctrlpts))
# Set evaluation delta
curve.delta = 0.01
curvepts = np.array(curve.evalpts)
# curveplt, = plt.plot(curvepts[:, 0], curvepts[:, 1], color='yellowgreen', linestyle='-')

trace2 = go.Scatter3d(
    x=curvepts[:, 0], 
    y=curvepts[:, 1], 
    z=curvepts[:, 2], 
    mode='lines',
    name='BSpline'
)

SToE = go.Scatter3d(
    x=[Start_point[0], Goal_point[0]], 
    y=[Start_point[1], Goal_point[1]],
    z=[Start_point[2], Goal_point[2]],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name='Start To End',
)

rv = multivariate_normal([10.1, 10.9, 10.2], cov)
out_pdf = rv.pdf(pos)
rv = multivariate_normal([6, 5, 6.5], cov)
out_pdf += rv.pdf(pos)
rv = multivariate_normal([7.3, 7.5, 9], cov)
out_pdf += rv.pdf(pos)
# rv = multivariate_normal(mean, cov)
# out_pdf = rv.pdf(pos)

gau = go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=out_pdf.flatten(),
    isomin=probability_threshold,
    # isomax=0.7,
    opacity=0.3,
    surface_count=32,
    # slices_z=dict(show=True, locations=[10.0]),
    surface=dict(fill=0.2, pattern='odd'), 
    caps= dict(x_show=False, y_show=False, z_show=False), # no caps
    colorscale='turbo') # color scale

layout = go.Layout(
    title='3D',
    # scene=dict(
    #         camera= dict(
    #             up=dict(x=0, y=0, z=1),
    #             center=dict(x=0, y=0, z=0),
    #             eye=dict(x=1, y=-1, z=0)
    #         )
    # )
)

fig = go.Figure(data=gau, layout=layout)
# fig.add_volume(gau)
# fig = go.Figure(data=[go.Scatter3d(x=curvepts[:, 0], y=curvepts[:, 1], z=curvepts[:, 2], mode='lines')], layout=layout)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(SToE)
fig.update_layout(height=800, width=960,
                  legend=dict(
                            orientation="h",
                            # entrywidth=500,
                            # yanchor="bottom",
                            # y=1.02,
                            # xanchor="right",
                            # x=1
                            )
                  )


plotly.offline.iplot(fig, filename='simple-3d-scatter')

# fig.show()
fig.write_html('3d_opt.html')

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly


# frames=[go.Frame(
#         data=[go.Scatter(
#             x=[xx[k]],
#             y=[yy[k]],
#             mode="markers",
#             marker=dict(color="red", size=10))])

#         for k in range(0, len(res.x), 3)]

# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Time:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
time = 0

# make frames
for i in range(0, len(x), 3):
    out = 0
    frame = {"data": [], "name": str(time)}
    for j in range(len(inturder_list)):
        # print(i, j)
        # gau = gauss_function_2d([x[i], x[i+1]], inturder_list[j][0][int(i/2)], inturder_list[j][1][int(i/2)], 1.05**(i/2), 1.05**(i/2))
        mean = np.array([inturder_list[j][0][int(i/3)], inturder_list[j][1][int(i/3)], inturder_list[j][2][int(i/3)]])
        rv = multivariate_normal(mean, cov)
        W = rv.pdf(pos)
        out = out + W
    
    data_dict = go.Volume(
                            x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                            value=out_pdf.flatten(),
                            isomin=probability_threshold,
                            # isomax=0.7,
                            opacity=0.3,
                            surface_count=32,
                            slices_z=dict(show=True, locations=[10.0]),
                            surface=dict(fill=0.2, pattern='odd'), 
                            caps= dict(x_show=False, y_show=False, z_show=False), # no caps
                            colorscale='turbo') # color scale    
    # tmp.append(out)
    frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)

slider_step = {"args": [
    [time],
    {"frame": {"duration": 300, "redraw": False},
        "mode": "immediate",
        "transition": {"duration": 300}}
                ],
    "label": time,
    "method": "animate"}
sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()






In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
import plotly.graph_objects as go

import numpy as np


# Generate curve data
t = np.linspace(-1, 1, 100)
x = t + t ** 2
y = t - t ** 2
xm = np.min(x) - 1.5
xM = np.max(x) + 1.5
ym = np.min(y) - 1.5
yM = np.max(y) + 1.5
N = 50
s = np.linspace(-1, 1, N)
xx = s + s ** 2
yy = s - s ** 2
vx = 1 + 2 * s
vy = 1 - 2 * s  # v=(vx, vy) is the velocity
speed = np.sqrt(vx ** 2 + vy ** 2)
ux = vx / speed  # (ux, uy) unit tangent vector, (-uy, ux) unit normal vector
uy = vy / speed

xend = xx + ux  # end coordinates for the unit tangent vector at (xx, yy)
yend = yy + uy

xnoe = xx - uy  # end coordinates for the unit normal vector at (xx,yy)
ynoe = yy + ux


# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     name="frame",
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     name="curve",
                     mode="lines",
                     line=dict(width=2, color="blue"))
          ],
    layout=go.Layout(width=600, height=600,
                     xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
                     yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
                     title="Moving Frenet Frame Along a Planar Curve",
                     hovermode="closest",
                     updatemenus=[dict(type="buttons",
                                       buttons=[dict(label="Play",
                                                     method="animate",
                                                     args=[None])])]),

    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k], xend[k], None, xx[k], xnoe[k]],
            y=[yy[k], yend[k], None, yy[k], ynoe[k]],
            mode="lines",
            line=dict(color="red", width=2))
        ]) for k in range(N)]
)

fig.show()

In [ ]:
import plotly.graph_objects as go

import numpy as np

# Generate curve data
t = np.linspace(-1, 1, 100)
x = t + t ** 2
y = t - t ** 2
xm = np.min(x) - 1.5
xM = np.max(x) + 1.5
ym = np.min(y) - 1.5
yM = np.max(y) + 1.5
N = 50
s = np.linspace(-1, 1, N)
xx = s + s ** 2
yy = s - s ** 2


# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
        yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k]],
            y=[yy[k]],
            mode="markers",
            marker=dict(color="red", size=10))])

        for k in range(N)]
)

fig.show()